<a href="https://colab.research.google.com/github/nehakalakonda/Ragam-Classification/blob/main/ragam_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [2]:
data = pd.read_csv('/content/raag.csv', encoding_errors='replace')

input_sequences = data['PHRASES'].values
labels = data['RAGAM'].values

label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(labels)
one_hot_labels = to_categorical(integer_encoded_labels)

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_sequences)
sequences = tokenizer.texts_to_sequences(input_sequences)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [4]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50
embeddings = Embedding(vocab_size, embedding_dim)

In [5]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(set(labels)), activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(padded_sequences, one_hot_labels, epochs=10, validation_split=0.2)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.5827 - loss: 1.0623 - val_accuracy: 0.0000e+00 - val_loss: 1.5929
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6053 - loss: 0.9195 - val_accuracy: 0.0000e+00 - val_loss: 1.6514
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6670 - loss: 0.7207 - val_accuracy: 0.2222 - val_loss: 1.0677
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7491 - loss: 0.6037 - val_accuracy: 0.4921 - val_loss: 0.9055
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8496 - loss: 0.4617 - val_accuracy: 0.6032 - val_loss: 0.7630
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9464 - loss: 0.2572 - val_accuracy: 0.8730 - val_loss: 0.3965
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9227 - loss: 0.2026 - val_accuracy: 0.9206 - val_loss: 0.2851
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9707 - loss: 0.1033 - val_accuracy: 0.9365 - val_loss:

In [7]:
loss, accuracy = model.evaluate(padded_sequences,one_hot_labels)
print(f'Test Accuracy: {accuracy}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9969 - loss: 0.0249     
Test Accuracy: 0.9903537034988403


In [8]:
single_input_sequence = "Ni Ri Pa Ma Ga Ri Ga Da Pa Ma Ga Ri Ga Ma Pa Ma Ga Ri Sa"

sequence = tokenizer.texts_to_sequences([single_input_sequence])
padded_sequence = pad_sequences(sequence, maxlen=max_length)

prediction = model.predict(padded_sequence)

predicted_label = list(set(labels))[prediction.argmax()]

print(f"Input Sequence: {single_input_sequence}")
print(f"Predicted Ragam: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Input Sequence: Ni Ri Pa Ma Ga Ri Ga Da Pa Ma Ga Ri Ga Ma Pa Ma Ga Ri Sa
Predicted Ragam: HAMSDHVANI


In [9]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00


In [10]:
import gradio as gr
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

data = pd.read_csv('/content/raag.csv', encoding_errors='replace')
input_sequences = data['PHRASES'].values
labels = data['RAGAM'].values

label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(labels)
one_hot_labels = to_categorical(integer_encoded_labels)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_sequences)
sequences = tokenizer.texts_to_sequences(input_sequences)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length)
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(set(labels)), activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(padded_sequences, one_hot_labels, epochs=10, validation_split=0.2)
loss, accuracy = model.evaluate(padded_sequences, one_hot_labels)
print(f'Test Accuracy: {accuracy}')

def predict_ragam(input_sequence):
    sequence = tokenizer.texts_to_sequences([input_sequence])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)
    prediction = model.predict(padded_sequence)
    predicted_label = label_encoder.inverse_transform([prediction.argmax()])[0]
    return predicted_label

interface = gr.Interface(
    fn=predict_ragam,
    inputs="text",
    outputs="text",
    title="Ragam Prediction",
    description="Enter a musical phrase to predict the corresponding ragam."
)

interface.launch()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.4331 - loss: 1.0764 - val_accuracy: 0.0000e+00 - val_loss: 1.5000
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6590 - loss: 0.8921 - val_accuracy: 0.0000e+00 - val_loss: 1.5138
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6600 - loss: 0.7188 - val_accuracy: 0.3333 - val_loss: 1.0233
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7564 - loss: 0.5738 - val_accuracy: 0.6508 - val_loss: 0.7937
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8997 - loss: 0.4088 - val_accuracy: 0.8413 - val_loss: 0.5901
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9398 - loss: 0.2551 - val_accuracy: 0.8571 - val_loss: 0.4360
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9653 - loss: 0.1400 - val_accuracy: 0.9206 - val_loss: 0.2203
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9656 - loss: 0.0864 - val_accuracy: 0.9365 - val_loss: